# Library

In [11]:
# Librerie standard
import os
import random
import time
import re
import shutil
from pathlib import Path
from collections import defaultdict, Counter
from itertools import islice
from concurrent.futures import ProcessPoolExecutor
import warnings

# Librerie per il trattamento delle immagini
import cv2
import imageio.v3 as imageio
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Librerie per il machine learning e deep learning
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as TF
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Librerie per la gestione dei dati
import pandas as pd
import json
import orjson
import ast

# Librerie per il progresso e il monitoraggio
from tqdm import tqdm

# Librerie per il deep learning avanzato
from torch.amp import GradScaler, autocast
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# Path

In [12]:
COCO_JSON_NM = 'COCO_annotations_new.json' 
OUT_COCO_JSON_NM = 'mod_COCO_annotations_new.json'
OUT_IMAGE_FLDR_NM = 'images'
RANDOM_SEED = 2023

in_dataset_pth = Path('/kaggle/input/our-xview-dataset')
out_dataset_pth = Path('/kaggle/working/')
img_fldr = Path(f'/kaggle/input/our-xview-dataset/{OUT_IMAGE_FLDR_NM}')

coco_json_pth = in_dataset_pth / COCO_JSON_NM
new_coco_json_pth = out_dataset_pth / OUT_COCO_JSON_NM

train_path = '/kaggle/working/train.json'
test_path = '/kaggle/working/test.json'
val_path = '/kaggle/working/val.json'

random.seed(RANDOM_SEED)

In [13]:
# Pulizia dell'output per cartelle specifiche
def clean_output(output_dir):
    if output_dir.exists() and output_dir.is_dir():
        for item in output_dir.iterdir():
            if item.is_dir():
                shutil.rmtree(item)  # Rimuove la sotto-cartella
            else:
                item.unlink()  # Rimuove il file
        print(f"Cartella {output_dir} pulita.")
    else:
        print(f"Cartella {output_dir} non trovata. Nessuna azione necessaria.")

# Pulisce la cartella di output prima di avviare il processo
clean_output(out_dataset_pth)

Cartella /kaggle/working pulita.


In [14]:
# Sopprime i warning specifici del modulo skimage
warnings.filterwarnings("ignore", 
    message="Applying `local_binary_pattern` to floating-point images may give unexpected results.*")

# Background Labels

In [15]:
def process_custom_coco_json(input_path, output_path):
    """
    Funzione per processare un JSON COCO in formato personalizzato.
    """
    # Leggi il JSON dal file di input
    with open(input_path, 'r') as f:
        data = json.load(f)

    # Ottieni e correggi il formato delle categorie
    raw_categories = data.get('categories', [])
    categories = []

    for category in tqdm(raw_categories, desc="Processing Categories"):
        for id_str, name in category.items():
            try:
                categories.append({"id": int(id_str), "name": name})
            except ValueError:
                print(f"Errore nel parsing della categoria: {category}")

    # Trova la categoria "Aircraft" con ID 0
    aircraft_category = next((cat for cat in categories if cat['id'] == 0 and cat['name'] == "Aircraft"), None)
    if aircraft_category:
        aircraft_category['id'] = 11  # Cambia l'ID della categoria "Aircraft" a 11

    # Aggiungi la categoria "background" con ID 0 se non esiste
    if not any(cat['id'] == 0 for cat in categories):
        categories.append({"id": 0, "name": "background"})

    # Preprocessa le annotazioni in un dizionario per immagini
    image_annotations_dict = {}
    for annotation in tqdm(data.get('annotations', []), desc="Building Image Annotations Dictionary"):
        image_id = annotation['image_id']
        if image_id not in image_annotations_dict:
            image_annotations_dict[image_id] = []
        image_annotations_dict[image_id].append(annotation)

    # Lista di nuove annotazioni da aggiungere per immagini senza bbox
    new_annotations = []

    # Elenco di annotazioni da rimuovere
    annotations_to_remove = []

    for annotation in tqdm(data.get('annotations', []), desc="Processing Annotations"):
        if annotation['category_id'] == 0:  # Se è Aircraft
            annotation['category_id'] = 11
        
        # Converte il formato del bbox
        if isinstance(annotation['bbox'], str):
            annotation['bbox'] = json.loads(annotation['bbox'])
        
        x, y, width, height = annotation['bbox']
        xmin = x
        xmax = x + width
        ymin = y
        ymax = y + height
        
        # Verifica che xmin < xmax e ymin < ymax
        if xmin >= xmax or ymin >= ymax:
            annotations_to_remove.append(annotation['id'])
        else:
            annotation['bbox'] = [xmin, xmax, ymin, ymax]

    # Rimuovi le annotazioni non valide
    data['annotations'] = [ann for ann in data['annotations'] if ann['id'] not in annotations_to_remove]

    # Verifica se ci sono immagini senza annotazioni (usando il dizionario delle annotazioni)
    for image in tqdm(data.get('images', []), desc="Processing Images"):
        if image['id'] not in image_annotations_dict:  # Se l'immagine non ha annotazioni
            # Aggiungi la categoria "background"
            new_annotation = {
                'id': len(data['annotations']) + len(new_annotations),
                'image_id': image['id'],
                'category_id': 0,  # Categoria background con ID 0
                'area': image['width'] * image['height'],
                'bbox': [0.0, image['width'], 0.0, image['height']],  # Background con bbox che copre tutta l'immagine
                'iscrowd': 0
            }
            new_annotations.append(new_annotation)

    # Aggiungi le nuove annotazioni al JSON originale
    data['annotations'].extend(new_annotations)

    # Aggiorna le categorie nel JSON
    data['categories'] = categories

    # Scrivi il JSON modificato nel file di output
    with open(output_path, 'w') as f:
        json.dump(data, f, indent=4)

In [16]:
process_custom_coco_json(coco_json_pth, new_coco_json_pth)

Processing Images: 100%|██████████| 45891/45891 [00:00<00:00, 723786.19it/s]


In [17]:
def count_bboxes_per_category(json_path):
    """
    Funzione che conta il numero di bounding box per ciascuna categoria in un file JSON formato COCO.
    
    :param json_path: Percorso al file JSON.
    :return: Dizionario con i nomi delle categorie come chiavi e il conteggio dei bounding box come valori.
    """
    # Leggi il JSON dal file
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Ottieni mapping delle categorie (id -> nome)
    category_mapping = {cat['id']: cat['name'] for cat in data.get('categories', [])}
    
    # Conta i bounding box per ciascun category_id
    bbox_counts = defaultdict(int)
    for annotation in data.get('annotations', []):
        category_id = annotation['category_id']
        bbox_counts[category_id] += 1
    
    # Converti il conteggio usando i nomi delle categorie
    bbox_counts_named = {category_mapping[cat_id]: count for cat_id, count in bbox_counts.items()}

    return bbox_counts_named

In [18]:
bbox_counts = count_bboxes_per_category(new_coco_json_pth)

# Stampa i risultati
for category, count in bbox_counts.items():
    print(f"Categoria: {category}, Numero di bbox: {count}")

Categoria: Passenger Vehicle, Numero di bbox: 224911
Categoria: Building, Numero di bbox: 384929
Categoria: Truck, Numero di bbox: 34345
Categoria: Engineering Vehicle, Numero di bbox: 5477
Categoria: Shipping Container, Numero di bbox: 5388
Categoria: Maritime Vessel, Numero di bbox: 6329
Categoria: Railway Vehicle, Numero di bbox: 4233
Categoria: Storage Tank, Numero di bbox: 2033
Categoria: Aircraft, Numero di bbox: 1708
Categoria: Pylon, Numero di bbox: 470
Categoria: Helipad, Numero di bbox: 152
Categoria: background, Numero di bbox: 13691


# Splitting

In [29]:
def filter_invalid_boxes(annotations):
    """Filtra le annotazioni con bounding box non validi."""
    valid_annotations = []
    for annotation in annotations:
        # Estrai le coordinate del bounding box
        bbox = annotation['bbox']
        if isinstance(bbox, str):
            try:
                # Tenta di convertire la stringa in una lista
                bbox = json.loads(bbox)
            except json.JSONDecodeError:
                raise ValueError(f"Bounding box non valido: {bbox} (conversione da stringa fallita).")

        # Verifica se il bounding box ha coordinate valide
        if len(bbox) == 4:
            x_min, y_min, width, height = bbox
            if width > 0 and height > 0:
                # Converti in formato [x_min, x_max, y_min, y_max]
                annotation['bbox'] = [x_min, x_min + width, y_min, y_min + height]
                valid_annotations.append(annotation)

    return valid_annotations


def split(json_file, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # Carica il JSON
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Filtra le annotazioni con bounding box non validi
    valid_annotations = filter_invalid_boxes(data['annotations'])
    
    # Ottieni la lista delle immagini
    images = data['images']
    
    # Mescola casualmente gli ID delle immagini
    random.shuffle(images)
    
    # Calcola i limiti per train, validation e test
    total_images = len(images)
    total_annotations = len(valid_annotations)
    train_end = int(total_images * train_ratio)
    val_end = int(total_images * (train_ratio + val_ratio))
    
    # Suddividi le immagini nei rispettivi set
    train_images = images[:train_end]
    val_images = images[train_end:val_end]
    test_images = images[val_end:]
    
    # Raggruppa gli ID delle immagini per i rispettivi set
    train_image_ids = {image['id'] for image in train_images}
    val_image_ids = {image['id'] for image in val_images}
    test_image_ids = {image['id'] for image in test_images}
    
    # Filtra le annotazioni per i rispettivi set di immagini
    train_annotations = [ann for ann in valid_annotations if ann['image_id'] in train_image_ids]
    val_annotations = [ann for ann in valid_annotations if ann['image_id'] in val_image_ids]
    test_annotations = [ann for ann in valid_annotations if ann['image_id'] in test_image_ids]
    
    # Crea i nuovi JSON per train, validation e test
    train_data = {'images': train_images, 'annotations': train_annotations, 'categories': data['categories']}
    val_data = {'images': val_images, 'annotations': val_annotations, 'categories': data['categories']}
    test_data = {'images': test_images, 'annotations': test_annotations, 'categories': data['categories']}
    
    # Salva i file JSON
    with open('train.json', 'w') as f:
        json.dump(train_data, f, indent=4)
    
    with open('val.json', 'w') as f:
        json.dump(val_data, f, indent=4)
    
    with open('test.json', 'w') as f:
        json.dump(test_data, f, indent=4)
    
    # Controlla la proporzione delle immagini e delle annotazioni
    check_split_proportions(total_images, total_annotations, 
                            len(train_images), len(val_images), len(test_images), 
                            len(train_annotations), len(val_annotations), len(test_annotations), 
                            train_ratio, val_ratio, test_ratio, 
                            train_annotations, val_annotations, test_annotations, data['categories'])


def check_split_proportions(total_images, total_annotations, train_count, val_count, test_count, 
                            train_bbox_count, val_bbox_count, test_bbox_count, 
                            train_ratio, val_ratio, test_ratio, 
                            train_annotations, val_annotations, test_annotations, categories):
    # Percentuali per immagini
    train_image_percentage = (train_count / total_images) * 100
    val_image_percentage = (val_count / total_images) * 100
    test_image_percentage = (test_count / total_images) * 100
    
    # Percentuali per bbox
    train_bbox_percentage = (train_bbox_count / total_annotations) * 100
    val_bbox_percentage = (val_bbox_count / total_annotations) * 100
    test_bbox_percentage = (test_bbox_count / total_annotations) * 100
    
    print(f"Totale immagini: {total_images}")
    print(f"Totale annotazioni (bbox): {total_annotations}")
    print(f"Train: {train_count} immagini ({train_image_percentage:.2f}%) ({train_bbox_count} bbox) ({train_bbox_percentage:.2f}%)")
    print(f"Val: {val_count} immagini ({val_image_percentage:.2f}%) ({val_bbox_count} bbox) ({val_bbox_percentage:.2f}%)")
    print(f"Test: {test_count} immagini ({test_image_percentage:.2f}%) ({test_bbox_count} bbox) ({test_bbox_percentage:.2f}%)")
    
    # Calcola il numero di annotazioni per categoria nei vari set
    category_count_train = defaultdict(int)
    category_count_val = defaultdict(int)
    category_count_test = defaultdict(int)
    
    for annotation in train_annotations:
        category_count_train[annotation['category_id']] += 1
    for annotation in val_annotations:
        category_count_val[annotation['category_id']] += 1
    for annotation in test_annotations:
        category_count_test[annotation['category_id']] += 1
    
    # Stampa le proporzioni per categoria
    print("\nProporzioni per categoria:")
    for category_dict in categories:
        for category_id, category_name in category_dict.items():
            # Converti category_id in intero se necessario
            category_id = int(category_id)
            
            # Conta il numero di annotazioni per categoria in ogni set
            train_cat_count = category_count_train.get(category_id, 0)
            val_cat_count = category_count_val.get(category_id, 0)
            test_cat_count = category_count_test.get(category_id, 0)
            
            # Calcola la percentuale di annotazioni per categoria
            total_cat_annotations = train_cat_count + val_cat_count + test_cat_count
            if total_cat_annotations > 0:
                train_cat_percentage = (train_cat_count / total_cat_annotations) * 100
                val_cat_percentage = (val_cat_count / total_cat_annotations) * 100
                test_cat_percentage = (test_cat_count / total_cat_annotations) * 100
            else:
                train_cat_percentage = val_cat_percentage = test_cat_percentage = 0.0

            print(f"{category_name}:")
            print(f"  Train: {train_cat_count} annotazioni ({train_cat_percentage:.2f}%)")
            print(f"  Val: {val_cat_count} annotazioni ({val_cat_percentage:.2f}%)")
            print(f"  Test: {test_cat_count} annotazioni ({test_cat_percentage:.2f}%)")

In [30]:
# Chiamata della funzione
split(coco_json_pth)

Totale immagini: 45891
Totale annotazioni (bbox): 669975
Train: 36712 immagini (80.00%) (533486 bbox) (79.63%)
Val: 4589 immagini (10.00%) (68191 bbox) (10.18%)
Test: 4590 immagini (10.00%) (69598 bbox) (10.39%)

Proporzioni per categoria:


# DataLoader

In [31]:
import operator
class CustomDataset(Dataset):
    def __init__(self, json_file, img_dir, aug=False):
        """
        Inizializza il dataset personalizzato.
        Args:
        - json_file: Il file JSON preprocessato contenente immagini, annotazioni e categorie.
        - img_dir: La directory contenente le immagini.
        - aug: Booleano per attivare o meno l'augmentazione.
        """
        # Carica il file JSON preprocessato
        with open(json_file, 'r') as f:
            coco_data = json.load(f)

        # Estrai informazioni su immagini, annotazioni e categorie
        self.image_info = {image['id']: image['file_name'] for image in coco_data['images']}
        self.image_annotations = {}
        self.image_bboxes = {}

        # Estrai le classi (categorie) dal file JSON
        self.classes = {}
        for category_dict in coco_data['categories']:
            for category_id, category_name in category_dict.items():
                # Converti category_id in intero se necessario
                self.classes[int(category_id)] = category_name

        for annotation in coco_data['annotations']:
            image_id = annotation['image_id']
            bbox = annotation['bbox']

            # Associa annotazioni e bounding box alle immagini
            if image_id not in self.image_annotations:
                self.image_annotations[image_id] = []
                self.image_bboxes[image_id] = []
            
            self.image_annotations[image_id].append(annotation['category_id'])
            self.image_bboxes[image_id].append(bbox)

        # Configura il percorso delle immagini e seleziona solo immagini valide
        self.img_dir = img_dir
        self.image_paths = []
        self.image_ids = []
        for image_id, file_name in self.image_info.items():
            if image_id in self.image_annotations:
                img_path = os.path.join(img_dir, file_name)
                if os.path.exists(img_path):
                    self.image_paths.append(img_path)
                    self.image_ids.append(image_id)

        # Definisci trasformazioni
        self.base_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

        self.aug_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

        self.aug = aug

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        """
        Recupera un'immagine e le sue annotazioni.
        """
        img_path = self.image_paths[index]
        img_id = self.image_ids[index]

        # Carica l'immagine
        image = Image.open(img_path).convert('RGB')
        original_width, original_height = image.size

        # Applica le trasformazioni
        if self.aug:
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)

        # Recupera le annotazioni e i bounding box
        categories = self.image_annotations[img_id]
        bboxes = self.image_bboxes[img_id]

        # Calcola il ridimensionamento
        scale_x = 224 / original_width
        scale_y = 224 / original_height

        # Ridimensiona i bounding box
        scaled_bboxes = [
            torch.tensor([
                bbox[0] * scale_x,  # x_min
                bbox[2] * scale_x,  # x_max
                bbox[1] * scale_y,  # y_min
                bbox[3] * scale_y   # y_max
            ], dtype=torch.float32)
            for bbox in bboxes
        ]

        # Costruisci il target
        target = {
            "boxes": torch.stack(scaled_bboxes),
            "labels": torch.tensor(categories, dtype=torch.int64)
        }

        return image_tensor, target

In [32]:
def collate_fn(batch):
    """
    Funzione di collation per il DataLoader, utile per il batching di immagini e annotazioni.
    La funzione restituirà un batch di immagini e un batch di target, formattato correttamente per Faster R-CNN.
    
    Args:
    - batch: lista di tuple (image, target)
    
    Returns:
    - images: batch di immagini
    - targets: lista di dizionari contenenti le annotazioni per ogni immagine
    """
    # Separa immagini e target
    images, targets = zip(*batch)

    # Converte la lista di immagini in un batch di immagini
    images = list(images)

    # Restituisci il batch
    return images, list(targets)

In [33]:
# Creazione dei dataset
train_dataset = CustomDataset(train_path, img_dir=img_fldr, aug=True)
valid_dataset = CustomDataset(val_path, img_dir=img_fldr, aug=False)  
test_dataset = CustomDataset(test_path, img_dir=img_fldr, aug=False)  

# Creazione dei DataLoader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn, num_workers=2, pin_memory=True)
val_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn, num_workers=2, pin_memory=True)

## Check DataLoader

In [34]:
def validate_dataloader(dataloader):
    """
    Valida un DataLoader verificando che ogni immagine abbia un target associato
    e che nessun target sia `None` o vuoto.
    
    Args:
    - dataloader: Il DataLoader da verificare.
    
    Returns:
    - error_messages: Lista di messaggi di errore. Vuota se tutti i dati sono validi.
    """
    error_messages = []
    for batch_idx, (images, targets) in enumerate(dataloader):
        for idx, target in enumerate(targets):
            if target is None:
                error_messages.append(f"Batch {batch_idx}, Immagine {idx}: Target è None.")
            elif target["boxes"].numel() == 0 or target["labels"].numel() == 0:
                error_messages.append(
                    f"Batch {batch_idx}, Immagine {idx}: Target è vuoto o mancano 'boxes'/'labels'."
                )
    return error_messages

In [35]:
# Validazione del DataLoader di training
train_errors = validate_dataloader(train_loader)

if train_errors:
    print("Errori nel DataLoader di training:")
    for error in train_errors:
        print(error)
else:
    print("Tutti i target nel DataLoader di training sono validi.")

Tutti i target nel DataLoader di training sono validi.


In [36]:
# Validazione del DataLoader di training
val_errors = validate_dataloader(val_loader)

if val_errors:
    print("Errori nel DataLoader di validation:")
    for error in val_errors:
        print(error)
else:
    print("Tutti i target nel DataLoader di validation sono validi.")

Tutti i target nel DataLoader di validation sono validi.


In [37]:
# Validazione del DataLoader di training
test_errors = validate_dataloader(test_loader)

if test_errors:
    print("Errori nel DataLoader di test:")
    for error in test_errors:
        print(error)
else:
    print("Tutti i target nel DataLoader di test sono validi.")

Tutti i target nel DataLoader di test sono validi.


In [38]:
def count_images_and_targets(dataloader):
    """
    Conta il numero totale di immagini e target in un DataLoader.
    """
    num_images = 0
    num_targets = 0

    for images, targets in dataloader:
        # Conta le immagini nel batch
        num_images += len(images)
        
        # Conta i target per ogni immagine (numero di oggetti)
        for target in targets:
            num_targets += len(target["boxes"])  # Ogni immagine ha un numero di bounding boxes
    
    return num_images, num_targets

In [39]:
num_images_train, num_targets_train = count_images_and_targets(train_loader)

print(f"Numero totale di immagini per il train: {num_images_train}")
print(f"Numero totale di target per il train: {num_targets_train}")

Numero totale di immagini per il train: 25781
Numero totale di target per il train: 533486


In [40]:
num_images_val, num_targets_val = count_images_and_targets(val_loader)

print(f"Numero totale di immagini per il validation: {num_images_val}")
print(f"Numero totale di target per il validation: {num_targets_val}")

Numero totale di immagini per il validation: 3204
Numero totale di target per il validation: 68191


In [41]:
num_images_test, num_targets_test = count_images_and_targets(test_loader)

print(f"Numero totale di immagini per il test: {num_images_test}")
print(f"Numero totale di target per il test: {num_targets_test}")

Numero totale di immagini per il test: 3195
Numero totale di target per il test: 69598


In [42]:
print(f"Numero totale di immagini: {num_images_train + num_images_val +num_images_test}")
print(f"Numero totale di target: {num_targets_train + num_targets_val +num_targets_test}")

Numero totale di immagini: 32180
Numero totale di target: 671275


# Modello Faster R-CNN (Resnet50)

In [43]:
def compute_class_weights(dataset):
    # Conta la frequenza di ogni classe nel dataset
    class_counts = np.zeros(len(dataset.classes))  
    
    # Usa tqdm per monitorare il progresso mentre si itera sul dataset
    for _, targets in tqdm(dataset, desc="Calcolo frequenze delle classi", leave=False):
        # Controlla se targets è None
        if targets is None:
            print("ERRORE TARGET NONE")
            continue
        
        # Assicurati che 'labels' sia un array e itera su di esso
        if 'labels' in targets:
            for target in targets['labels']:  
                class_counts[target] += 1

    # Calcola i pesi per le classi 
    total_count = sum(class_counts)  

    # Calcola i pesi inversamente proporzionali alla frequenza
    class_weights = np.divide(total_count, class_counts)

    return class_weights

def calculate_map(preds, targets, num_classes, thresholds=(0.4, 0.5, 0.6), areas=(32, 96)):
    """
    Calcola la mean Average Precision (mAP) con soglia di IoU adattiva per più classi.

    Args:
        preds (list): Liste di predizioni. Ogni elemento è un dizionario con chiavi:
            - "boxes": array (x1, y1, x2, y2) delle bounding box predette
            - "scores": array dei confidence scores
            - "labels": array delle classi predette
        targets (list): Liste di ground truth. Ogni elemento è un dizionario con chiavi:
            - "boxes": array (x1, y1, x2, y2) delle bounding box reali
            - "labels": array delle classi reali
        num_classes (int): Numero totale di classi.
        thresholds (tuple): Soglie di IoU per oggetti piccoli, medi e grandi.
        areas (tuple): Limiti per classificare gli oggetti in "piccoli", "medi" e "grandi".

    Returns:
        float: Mean Average Precision (mAP) calcolata con soglia adattiva.
    """
    def compute_iou(box1, box2):
        """Calcola l'IoU tra due bounding boxes."""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])

        inter_area = max(0, x2 - x1) * max(0, y2 - y1)
        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union_area = box1_area + box2_area - inter_area

        return inter_area / union_area if union_area > 0 else 0

    def get_adaptive_threshold(area, thresholds, areas):
        """Determina la soglia di IoU adattiva in base all'area dell'oggetto."""
        small_threshold, medium_threshold, large_threshold = thresholds
        small_area, medium_area = areas

        if area < small_area:
            return small_threshold
        elif area < medium_area:
            return medium_threshold
        else:
            return large_threshold

    aps = []  # Average Precision per classe

    # Aggiungi tqdm per monitorare il ciclo per classe
    for cls in tqdm(range(num_classes), desc="Calcolando mAP per le classi"):
        cls_gt_boxes = []
        cls_pred_boxes = []
        cls_scores = []

        # Estrai bounding box e predizioni per la classe corrente
        for i in range(len(targets)):
            # Ground truth
            gt_boxes = targets[i]["boxes"]
            gt_labels = targets[i]["labels"]
            cls_gt_boxes.extend([gt_boxes[j] for j in range(len(gt_labels)) if gt_labels[j] == cls])

            # Predizioni
            pred_boxes = preds[i]["boxes"]
            pred_scores = preds[i]["scores"]
            pred_labels = preds[i]["labels"]
            for j in range(len(pred_labels)):
                if pred_labels[j] == cls:
                    cls_pred_boxes.append(pred_boxes[j])
                    cls_scores.append(pred_scores[j])

        # Ordina le predizioni per confidence score decrescente
        cls_pred_boxes = [x for _, x in sorted(zip(cls_scores, cls_pred_boxes), key=lambda pair: pair[0], reverse=True)]

        tp = np.zeros(len(cls_pred_boxes))
        fp = np.zeros(len(cls_pred_boxes))
        gt_used = np.zeros(len(cls_gt_boxes))

        # Aggiungi tqdm per monitorare il ciclo per ogni predizione
        for j, pred in enumerate(cls_pred_boxes):
            best_iou = 0
            best_gt_idx = -1

            for k, gt_box in enumerate(cls_gt_boxes):
                iou = compute_iou(pred, gt_box)
                area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
                adaptive_threshold = get_adaptive_threshold(area, thresholds, areas)

                if iou > best_iou and iou >= adaptive_threshold and not gt_used[k]:
                    best_iou = iou
                    best_gt_idx = k

            if best_gt_idx >= 0:
                tp[j] = 1
                gt_used[best_gt_idx] = 1
            else:
                fp[j] = 1

        # Calcola Precision e Recall
        cum_tp = np.cumsum(tp)
        cum_fp = np.cumsum(fp)
        precision = cum_tp / (cum_tp + cum_fp + 1e-6)
        recall = cum_tp / len(cls_gt_boxes) if len(cls_gt_boxes) > 0 else np.zeros_like(cum_tp)

        # Interpolazione per l'AP
        ap = 0
        for t in np.linspace(0, 1, 101):
            precisions = precision[recall >= t]
            ap += max(precisions) if len(precisions) > 0 else 0
        ap /= 101

        aps.append(ap)

    return np.mean(aps)

In [44]:
def calculate_map(preds, targets, num_classes, thresholds=(0.4, 0.5, 0.6), areas=(32, 96)):
    def compute_iou(box1, box2):
        """Calcola l'IoU tra due bounding boxes."""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])

        inter_area = max(0, x2 - x1) * max(0, y2 - y1)
        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union_area = box1_area + box2_area - inter_area

        return inter_area / union_area if union_area > 0 else 0

    def get_adaptive_threshold(area, thresholds, areas):
        """Determina la soglia di IoU adattiva in base all'area dell'oggetto."""
        small_threshold, medium_threshold, large_threshold = thresholds
        small_area, medium_area = areas

        if area < small_area:
            return small_threshold
        elif area < medium_area:
            return medium_threshold
        else:
            return large_threshold

    aps = []  # Average Precision per classe

    # Aggiungi tqdm per monitorare il ciclo per classe
    for cls in tqdm(range(num_classes), desc="Calcolando mAP per le classi"):
        cls_gt_boxes = []
        cls_pred_boxes = []
        cls_scores = []

        # Estrai bounding box e predizioni per la classe corrente
        for i in range(len(targets)):
            # Ground truth
            gt_boxes = targets[i]["boxes"]
            gt_labels = targets[i]["labels"]
            cls_gt_boxes.extend([gt_boxes[j] for j in range(len(gt_labels)) if gt_labels[j] == cls])

            # Predizioni
            pred_boxes = preds[i]["boxes"]
            pred_scores = preds[i]["scores"]
            pred_labels = preds[i]["labels"]
            for j in range(len(pred_labels)):
                if pred_labels[j] == cls:
                    cls_pred_boxes.append(pred_boxes[j])
                    cls_scores.append(pred_scores[j])

        # Ordina le predizioni per confidence score decrescente (utilizzando np.argsort per efficienza)
        sorted_idx = np.argsort(cls_scores)[::-1]
        cls_pred_boxes = np.array(cls_pred_boxes)[sorted_idx]
        cls_scores = np.array(cls_scores)[sorted_idx]

        tp = np.zeros(len(cls_pred_boxes))
        fp = np.zeros(len(cls_pred_boxes))
        gt_used = np.zeros(len(cls_gt_boxes))

        # Aggiungi tqdm per monitorare il ciclo per ogni predizione
        for j, pred in enumerate(cls_pred_boxes):
            best_iou = 0
            best_gt_idx = -1

            for k, gt_box in enumerate(cls_gt_boxes):
                iou = compute_iou(pred, gt_box)
                area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
                adaptive_threshold = get_adaptive_threshold(area, thresholds, areas)

                if iou > best_iou and iou >= adaptive_threshold and not gt_used[k]:
                    best_iou = iou
                    best_gt_idx = k

            if best_gt_idx >= 0:
                tp[j] = 1
                gt_used[best_gt_idx] = 1
            else:
                fp[j] = 1

        # Calcola Precision e Recall
        cum_tp = np.cumsum(tp)
        cum_fp = np.cumsum(fp)
        precision = cum_tp / (cum_tp + cum_fp + 1e-6)
        recall = cum_tp / len(cls_gt_boxes) if len(cls_gt_boxes) > 0 else np.zeros_like(cum_tp)

        # Interpolazione per l'AP
        ap = 0
        for t in np.linspace(0, 1, 101):
            precisions = precision[recall >= t]
            ap += max(precisions) if len(precisions) > 0 else 0
        ap /= 101

        aps.append(ap)

    return np.mean(aps)

In [45]:
def train_and_validate(model, train_loader, val_loader, optimizer, device, class_weights=None, 
                       num_epochs=10, num_classes=12, accumulation_steps=4):
    """
    Funzione di training e validazione per Faster R-CNN.
    """
    # Scaler per mixed precision training
    scaler = GradScaler()
    model.to(device)

    train_losses = []
    val_losses = []
    train_mAPs = []
    val_mAPs = []

    # Converte class_weights in tensor e sposta sul dispositivo
    if class_weights is not None:
        class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

    for epoch in range(num_epochs):
        print(f"\nEpoca {epoch + 1}/{num_epochs}")

        # --------------------
        # Training
        # --------------------
        model.train()
        total_train_loss = 0.0
        all_train_preds = []
        all_train_targets = []

        optimizer.zero_grad()
        train_loop = tqdm(train_loader, desc="Training", leave=False)

        for i, (images, targets) in enumerate(train_loop):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
            with autocast('cuda'):
                # Forward pass
                loss_dict = model(images, targets)
            
                total_weighted_loss = 0
                total_weight = 0
            
                # Itera su tutte le perdite
                for key, loss in loss_dict.items():
                    if isinstance(loss, torch.Tensor):
                        if key == 'loss_classifier' and class_weights is not None:
                            # Applica i pesi delle classi solo alla loss_classifier
                            labels = torch.cat([t['labels'] for t in targets])  # Ottieni le etichette
                            weighted_loss = loss * class_weights[labels]  # Pesi basati sulle etichette
                            total_weighted_loss += weighted_loss.sum()  # Somma la perdita pesata
                            total_weight += class_weights[labels].sum()  # Somma i pesi per il calcolo totale
                        else:
                            # Entra qui per loss_box_reg, loss_objectness e loss_rpn_box_reg
                            total_weighted_loss += loss.sum()  # Somma la perdita normale
                            total_weight += loss.numel()  # Somma il numero di elementi
            
                losses = total_weighted_loss / total_weight  # Calcola la perdita totale normalizzata

            # Backward pass con gradient scaling
            scaler.scale(losses).backward()
        
            # Gradient accumulation
            if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
        
            total_train_loss += losses.item() * accumulation_steps
            train_loop.set_postfix(loss=losses.item() * accumulation_steps)
            
            # Raccoglie predizioni per calcolo mAP
            model.eval()
            with torch.no_grad():
                outputs = model(images)
                all_train_preds.extend([{k: v.cpu() for k, v in t.items()} for t in outputs])
                all_train_targets.extend([{k: v.cpu() for k, v in t.items()} for t in targets])
            model.train()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"Perdita media di training: {avg_train_loss:.4f}")

        train_map = calculate_map(all_train_preds, all_train_targets, num_classes=num_classes)
        train_mAPs.append(train_map)
        print(f"mAP di training: {train_map:.4f}")
        
        # --------------------
        # Validazione
        # --------------------
        model.eval()
        total_val_loss = 0.0
        all_val_preds = []
        all_val_targets = []
        
        val_loop = tqdm(val_loader, desc="Validazione", leave=False)
        with torch.no_grad():
            for images, targets in val_loop:
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
                with autocast('cuda'):
                    # Forward pass
                    loss_dict = model(images, targets)
        
                    total_weighted_loss = 0
                    total_weight = 0
    
                    for single_loss_dict in loss_dict:
                        for i, loss in enumerate(single_loss_dict.values()):
                            if isinstance(loss, torch.Tensor):
                                weighted_loss = loss * class_weights[i]
                                total_weighted_loss += weighted_loss.sum()
                                total_weight += class_weights[i] * loss.numel()
    
                    losses = total_weighted_loss / total_weight
        
                total_val_loss += losses.item()
        
                # Predizioni per calcolo mAP
                outputs = model(images)
                all_val_preds.extend([{k: v.cpu() for k, v in t.items()} for t in outputs])
                all_val_targets.extend([{k: v.cpu() for k, v in t.items()} for t in targets])
        
        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Perdita media di validazione: {avg_val_loss:.4f}")
        
        val_map = calculate_map(all_val_preds, all_val_targets, num_classes=num_classes)
        val_mAPs.append(val_map)
        print(f"mAP di validazione: {val_map:.4f}")

        # --------------------
        # Salvataggio del modello
        # --------------------
        torch.save(model.state_dict(), f"model_epoch_{epoch + 1}.pth")
        print(f"Modello salvato: model_epoch_{epoch + 1}.pth")

    return train_losses, val_losses, train_mAPs, val_mAPs

In [46]:
def plot_metrics(train_losses, val_losses, train_mAPs, val_mAPs, num_epochs):
    """
    Funzione per plottare le metriche di training e validazione (Loss e mAP).
    """
    epochs_range = range(1, num_epochs + 1)

    plt.figure(figsize=(12, 8))

    # Plot della Loss
    plt.subplot(2, 1, 1)
    plt.plot(epochs_range, train_losses, label='Training Loss', color='blue')
    plt.plot(epochs_range, val_losses, label='Validation Loss', color='red')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss per Epoca')

    # Plot del mAP
    plt.subplot(2, 1, 2)
    plt.plot(epochs_range, train_mAPs, label='Training mAP', color='green')
    plt.plot(epochs_range, val_mAPs, label='Validation mAP', color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('mAP')
    plt.legend()
    plt.title('mAP per Epoca')

    plt.tight_layout()
    plt.show()

In [47]:
def visualize_predictions(image, boxes, labels, scores, class_names, threshold=0.5):
    """
    Visualizza le predizioni (bounding boxes e etichette) su una singola immagine.
    
    Args:
    - image: immagine di input.
    - boxes: liste delle bounding boxes (x1, y1, x2, y2).
    - labels: etichette delle predizioni.
    - scores: punteggi delle predizioni.
    - class_names: lista dei nomi delle classi.
    - threshold: punteggio minimo per visualizzare la predizione.
    """
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)

    # Filtriamo le predizioni che hanno un punteggio superiore alla soglia
    for box, label, score in zip(boxes, labels, scores):
        if score >= threshold:
            # Crea il rettangolo per la bounding box
            rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                     linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            
            # Aggiungi il label e il punteggio sopra la bounding box
            ax.text(box[0], box[1] - 10, f'{class_names[label]}: {score:.2f}', color='r',
                    fontsize=12, fontweight='bold', backgroundcolor='white')
    
    plt.show()

def test_model(model, test_loader, device, class_names, calculate_map=None, num_classes=12, num_visualizations=5):
    """
    Funzione per il testing del modello Faster R-CNN, con calcolo opzionale del mAP e visualizzazione delle predizioni.
    
    Args:
    - model: il modello Faster R-CNN.
    - test_loader: DataLoader per il test set.
    - device: dispositivo su cui eseguire (es. 'cuda' o 'cpu').
    - class_names: lista dei nomi delle classi.
    - calculate_map: funzione per calcolare il mAP (opzionale).
    - num_classes: numero di classi nel modello (default: 12).
    - num_visualizations: numero di immagini da visualizzare durante il test.
    
    Returns:
    - predictions: lista delle predizioni per ogni batch (include 'boxes', 'labels', 'scores').
    - mAP: valore medio del mAP se `calculate_map` è fornito.
    """
    model.to(device)
    model.eval()
    predictions = []
    all_preds = []  # Raccolta delle predizioni per il calcolo mAP
    all_targets = []  # Raccolta dei target per il calcolo mAP

    print("\nInizio testing...")
    test_loop = tqdm(test_loader, desc="Testing", leave=False)

    # Variabile per visualizzare le immagini
    visualized = 0

    with torch.no_grad():
        for images, targets in test_loop:  # Durante il test, i target sono necessari per calcolare il mAP
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]  # Prepariamo i target
            
            preds = model(images)
            
            # Predizioni di ciascun batch (contenente 'boxes', 'labels', 'scores')
            for pred, target in zip(preds, targets):
                predictions.append({
                    'boxes': pred['boxes'].cpu().numpy(),
                    'labels': pred['labels'].cpu().numpy(),
                    'scores': pred['scores'].cpu().numpy()
                })
                
                # Raccogliamo predizioni e target per calcolare il mAP
                all_preds.append(pred)
                all_targets.append(target)

                # Visualizzare le predizioni su una immagine
                if visualized < num_visualizations:
                    visualize_predictions(
                        images[0].cpu().numpy().transpose(1, 2, 0),  # Converti l'immagine in formato (H, W, C)
                        pred['boxes'].cpu().numpy(),
                        pred['labels'].cpu().numpy(),
                        pred['scores'].cpu().numpy(),
                        class_names
                    )
                    visualized += 1

            # Aggiungi aggiornamenti su quante predizioni sono state processate
            test_loop.set_postfix(processed=len(predictions))

    # Calcolo del mAP se la funzione di calcolo è stata fornita
    mAP = None
    if calculate_map is not None:
        mAP = calculate_map(all_preds, all_targets, num_classes=num_classes)
        print(f"mAP di testing: {mAP:.4f}")

    print("Testing completato.")
    return predictions, mAP

In [48]:
# Carica il modello Faster R-CNN con ResNet50 e FPN
model = fasterrcnn_resnet50_fpn(weights=None)

num_classes = 12

# Modifica il numero di classi in output
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Congela i layer della backbone (ResNet50)
for param in model.backbone.parameters():
    param.requires_grad = False

# Imposta il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Configurazione training
num_epochs = 2
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# Sposta il modello su GPU o CPU
model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 186MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [49]:
# Calcola i pesi delle classi
class_weights = compute_class_weights(train_loader.dataset)

print(class_weights)

[3.89975146e+02 2.99420226e+00 1.94348270e+01 1.57649527e+02
 1.07514309e+02 1.25526118e+02 1.73551772e+00 4.16785938e+03
 3.21183624e+02 1.22753336e+02 1.43796765e+03]


In [ ]:
train_losses, val_losses, train_mAPs, val_mAPs = train_and_validate(model, train_loader, val_loader, optimizer, device, class_weights, num_epochs)


Epoca 1/2


Training:  22%|██▏       | 2803/12891 [15:02<53:51,  3.12it/s, loss=1.16]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Perdita media di training: 1.3757


Calcolando mAP per le classi:   8%|▊         | 1/12 [00:19<03:39, 19.96s/it]

In [ ]:
plot_metrics(train_losses, val_losses, train_mAPs, val_mAPs, num_epochs)

In [ ]:
predictions, mAP = test_model(model, test_loader, device, class_names=class_names, calculate_map=calculate_map, num_classes=num_classes, num_visualizations=5)